In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_openai import AzureChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [4]:
llm  = AzureChatOpenAI(azure_deployment='gpt-4o',temperature=0)

In [5]:
# Import Azure OpenAI
from langchain_openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [60]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [8]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [29]:
from langchain_core.output_parsers.json import JsonOutputParser
quiz_chain1 = quiz_generation_prompt | llm | JsonOutputParser()

In [9]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\JAGADEESHNAIKPALTHYA\AppData\Local\Temp\ipykernel_16416\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [10]:
TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [11]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [30]:
from langchain_core.output_parsers.json import JsonOutputParser
review_chain1 = quiz_evaluation_prompt | llm | JsonOutputParser()

In [12]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [32]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [14]:
from pathlib import Path
file_path=r"C:\Users\JAGADEESHNAIKPALTHYA\mcqgen1\data.txt"

In [15]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [55]:
NUMBER=5
SUBJECT="data science"
TONE="simple"

In [53]:
RESPONSE_JSON=json.dumps(RESPONSE_JSON)

In [ ]:
json.dumps(RESPONSE_JSON)

In [33]:
from langchain.callbacks import get_openai_callback

In [35]:
"""mcq_count=5
grade=1
tone="simple"""

'mcq_count=5\ngrade=1\ntone="simple'

In [61]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": RESPONSE_JSON
        }
        )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for data science students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
```json
{
  "1": {
    "mcq": "Who coined the term 'machine learning'?",
    "options": {
      "a": "Donald Hebb",
      "b": "Arthur Samuel",
      "c": "Walter Pitts",
      "d": "Tom M. Mitchell"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
    "options": {
      "a": "Cybertron",
      "b": "Deep Blue",
      "c": "Perceptron",
      "d": "AlphaGo"
    },
    "cor

In [67]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2189
Prompt Tokens:1429
Completion Tokens:760
Total Cost:0.011172500000000002


In [63]:
quiz_str = response.get('quiz')

In [66]:
quiz_str

'```json\n{\n  "1": {\n    "mcq": "Who coined the term \'machine learning\'?",\n    "options": {\n      "a": "Donald Hebb",\n      "b": "Arthur Samuel",\n      "c": "Walter Pitts",\n      "d": "Tom M. Mitchell"\n    },\n    "correct": "b"\n  },\n  "2": {\n    "mcq": "What was the name of the experimental \'learning machine\' developed by Raytheon Company in the early 1960s?",\n    "options": {\n      "a": "Cybertron",\n      "b": "Deep Blue",\n      "c": "Perceptron",\n      "d": "AlphaGo"\n    },\n    "correct": "a"\n  },\n  "3": {\n    "mcq": "Which book by Donald Hebb introduced a theoretical neural structure?",\n    "options": {\n      "a": "The Organization of Behavior",\n      "b": "Learning Machines",\n      "c": "Artificial Intelligence: A Modern Approach",\n      "d": "Pattern Recognition and Machine Learning"\n    },\n    "correct": "a"\n  },\n  "4": {\n    "mcq": "What is one of the main objectives of modern-day machine learning?",\n    "options": {\n      "a": "To replace h

In [65]:
json.loads(quiz_str)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [45]:
print("quiz_str:", quiz_str)

quiz_str: ```json
{
  "1": {
    "mcq": "Who coined the term 'machine learning' in 1959?",
    "options": {
      "a": "Donald Hebb",
      "b": "Arthur Samuel",
      "c": "Walter Pitts",
      "d": "Tom M. Mitchell"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
    "options": {
      "a": "Cybertron",
      "b": "Deep Blue",
      "c": "Perceptron",
      "d": "AlphaGo"
    },
    "correct": "a"
  },
  "3": {
    "mcq": "Which book by Donald Hebb introduced a theoretical neural structure?",
    "options": {
      "a": "The Organization of Behavior",
      "b": "Learning Machines",
      "c": "Artificial Intelligence",
      "d": "Neural Networks"
    },
    "correct": "a"
  },
  "4": {
    "mcq": "What is one of the main objectives of modern-day machine learning?",
    "options": {
      "a": "To replace human intelligence",
      "b": "To classify data based on models",

In [24]:
quiz_str=response.get("quiz")

In [46]:
quiz_table_data = []
for key, value in quiz_str.item():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

AttributeError: 'str' object has no attribute 'item'